In [1]:
import os
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.applications import MobileNetV2, EfficientNetB0
from tensorflow.keras.models import Model
from sklearn.metrics import classification_report, confusion_matrix

In [2]:
# Define paths
train_dir = "dataset/train"
test_dir = "dataset/test"

# Data augmentation for training data
train_datagen = ImageDataGenerator(
    rescale=1./255,         # Normalize pixel values
    rotation_range=30,      # Random rotation
    width_shift_range=0.2,  # Random horizontal shift
    height_shift_range=0.2, # Random vertical shift
    shear_range=0.2,        # Shearing
    zoom_range=0.2,         # Zooming
    horizontal_flip=True,   # Flip horizontally
    fill_mode='nearest'     # Fill missing pixels
)

# Preprocessing for testing data
test_datagen = ImageDataGenerator(rescale=1./255)

# Load the datasets
train_data = train_datagen.flow_from_directory(
    train_dir,
    target_size=(150, 150),
    batch_size=32,
    class_mode='binary'
)

test_data = test_datagen.flow_from_directory(
    test_dir,
    target_size=(150, 150),
    batch_size=32,
    class_mode='binary'
)


Found 1840 images belonging to 2 classes.
Found 460 images belonging to 2 classes.


In [3]:
cnn_model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(150, 150, 3)),
    MaxPooling2D((2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(1, activation='sigmoid')
])

# Compile the model
cnn_model.compile(optimizer=Adam(learning_rate=0.001),
                  loss='binary_crossentropy',
                  metrics=['accuracy'])

# Train the model
cnn_history = cnn_model.fit(
    train_data,
    epochs=10,
    validation_data=test_data
)


C:\Users\hamza\AppData\Roaming\Python\Python312\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
C:\Users\hamza\AppData\Roaming\Python\Python312\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/10
58/58 ━━━━━━━━━━━━━━━━━━━━ 48s 770ms/step - accuracy: 0.5106 - loss: 0.9831 - val_accuracy: 0.5500 - val_loss: 0.6855
Epoch 2/10
58/58 ━━━━━━━━━━━━━━━━━━━━ 42s 723ms/step - accuracy: 0.5713 - loss: 0.6714 - val_accuracy: 0.6565 - val_loss: 0.6399
Epoch 3/10
58/58 ━━━━━━━━━━━━━━━━━━━━ 44s 754ms/step - accuracy: 0.6639 - loss: 0.6423 - val_accuracy: 0.6261 - val_loss: 0.6720
Epoch 4/10
58/58 ━━━━━━━━━━━━━━━━━━━━ 45s 767ms/step - accuracy: 0.6596 - loss: 0.6374 - val_accuracy: 0.6891 - val_loss: 0.6238
Epoch 5/10
58/58 ━━━━━━━━━━━━━━━━━━━━ 41s 712ms/step - accuracy: 0.6583 - loss: 0.6291 - val_accuracy: 0.6739 - val_loss: 0.6253
Epoch 6/10
58/58 ━━━━━━━━━━━━━━━━━━━━ 45s 767ms/step - accuracy: 0.6575 - loss: 0.6149 - val_accuracy: 0.6609 - val_loss: 0.6397
Epoch 7/10
58/58 ━━━━━━━━━━━━━━━━━━━━ 42s 722ms/step - accuracy: 0.6512 - loss: 0.6177 - val_accuracy: 0.6826 - val_loss: 0.5998
Epoch 8/10
58/58 ━━━━━━━━━━━━━━━━━━━━ 41s 708ms/step - accuracy: 0.6951 - loss: 0.5992 - val_accu

In [4]:
# Load pre-trained MobileNetV2
base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(150, 150, 3))
base_model.trainable = False  # Freeze the base model

# Add custom layers
x = base_model.output
x = tf.keras.layers.GlobalAveragePooling2D()(x)
x = Dense(128, activation='relu')(x)
x = Dropout(0.5)(x)
output = Dense(1, activation='sigmoid')(x)

mobilenet_model = Model(inputs=base_model.input, outputs=output)

# Compile the model
mobilenet_model.compile(optimizer=Adam(learning_rate=0.0001),
                        loss='binary_crossentropy',
                        metrics=['accuracy'])

# Train the model
mobilenet_history = mobilenet_model.fit(
    train_data,
    epochs=10,
    validation_data=test_data
)


C:\Users\hamza\AppData\Local\Temp\ipykernel_8040\2309857045.py:2: UserWarning: `input_shape` is undefined or non-square, or `rows` is not in [96, 128, 160, 192, 224]. Weights for input shape (224, 224) will be loaded as the default.
  base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(150, 150, 3))


9406464/9406464 ━━━━━━━━━━━━━━━━━━━━ 4s 0us/step
Epoch 1/10
58/58 ━━━━━━━━━━━━━━━━━━━━ 49s 716ms/step - accuracy: 0.5696 - loss: 0.8417 - val_accuracy: 0.7739 - val_loss: 0.4532
Epoch 2/10
58/58 ━━━━━━━━━━━━━━━━━━━━ 41s 713ms/step - accuracy: 0.7561 - loss: 0.5618 - val_accuracy: 0.7891 - val_loss: 0.4214
Epoch 3/10
58/58 ━━━━━━━━━━━━━━━━━━━━ 42s 719ms/step - accuracy: 0.7934 - loss: 0.4544 - val_accuracy: 0.7848 - val_loss: 0.4170
Epoch 4/10
58/58 ━━━━━━━━━━━━━━━━━━━━ 42s 718ms/step - accuracy: 0.8056 - loss: 0.4220 - val_accuracy: 0.7978 - val_loss: 0.4005
Epoch 5/10
58/58 ━━━━━━━━━━━━━━━━━━━━ 39s 680ms/step - accuracy: 0.8124 - loss: 0.4159 - val_accuracy: 0.8087 - val_loss: 0.3724
Epoch 6/10
58/58 ━━━━━━━━━━━━━━━━━━━━ 40s 683ms/step - accuracy: 0.8089 - loss: 0.4123 - val_accuracy: 0.8109 - val_loss: 0.3687
Epoch 7/10
58/58 ━━━━━━━━━━━━━━━━━━━━ 39s 678ms/step - accuracy: 0.8229 - loss: 0.4122 - val_accuracy: 0.8130 - val_loss: 0.3700
Epoch 8/10
58/58 ━━━━━━━━━━━━━━━━━━━━ 40s 686ms/

In [5]:
# Load pre-trained MobileNetV2
base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(150, 150, 3))
base_model.trainable = False  # Freeze the base model

# Add custom layers
x = base_model.output
x = tf.keras.layers.GlobalAveragePooling2D()(x)
x = Dense(128, activation='relu')(x)
x = Dropout(0.5)(x)
output = Dense(1, activation='sigmoid')(x)

mobilenet_model = Model(inputs=base_model.input, outputs=output)

# Compile the model
mobilenet_model.compile(optimizer=Adam(learning_rate=0.0001),
                        loss='binary_crossentropy',
                        metrics=['accuracy'])

# Train the model
mobilenet_history = mobilenet_model.fit(
    train_data,
    epochs=25,
    validation_data=test_data
)

C:\Users\hamza\AppData\Local\Temp\ipykernel_8040\3703232714.py:2: UserWarning: `input_shape` is undefined or non-square, or `rows` is not in [96, 128, 160, 192, 224]. Weights for input shape (224, 224) will be loaded as the default.
  base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(150, 150, 3))


Epoch 1/25
58/58 ━━━━━━━━━━━━━━━━━━━━ 49s 726ms/step - accuracy: 0.5847 - loss: 0.8495 - val_accuracy: 0.7435 - val_loss: 0.4884
Epoch 2/25
58/58 ━━━━━━━━━━━━━━━━━━━━ 40s 690ms/step - accuracy: 0.7581 - loss: 0.5262 - val_accuracy: 0.7870 - val_loss: 0.4408
Epoch 3/25
58/58 ━━━━━━━━━━━━━━━━━━━━ 39s 675ms/step - accuracy: 0.7883 - loss: 0.4710 - val_accuracy: 0.7804 - val_loss: 0.4393
Epoch 4/25
58/58 ━━━━━━━━━━━━━━━━━━━━ 39s 670ms/step - accuracy: 0.8012 - loss: 0.4353 - val_accuracy: 0.8304 - val_loss: 0.3758
Epoch 5/25
58/58 ━━━━━━━━━━━━━━━━━━━━ 39s 663ms/step - accuracy: 0.7923 - loss: 0.4242 - val_accuracy: 0.8348 - val_loss: 0.3687
Epoch 6/25
58/58 ━━━━━━━━━━━━━━━━━━━━ 39s 666ms/step - accuracy: 0.8172 - loss: 0.4082 - val_accuracy: 0.8500 - val_loss: 0.3504
Epoch 7/25
58/58 ━━━━━━━━━━━━━━━━━━━━ 39s 674ms/step - accuracy: 0.8352 - loss: 0.3987 - val_accuracy: 0.8152 - val_loss: 0.3832
Epoch 8/25
58/58 ━━━━━━━━━━━━━━━━━━━━ 39s 664ms/step - accuracy: 0.8419 - loss: 0.3939 - val_accu

In [6]:
# Load pre-trained EfficientNetB0
base_model = EfficientNetB0(weights='imagenet', include_top=False, input_shape=(150, 150, 3))
base_model.trainable = False  # Freeze the base model

# Add custom layers
x = base_model.output
x = tf.keras.layers.GlobalAveragePooling2D()(x)
x = Dense(128, activation='relu')(x)
x = Dropout(0.5)(x)
output = Dense(1, activation='sigmoid')(x)

efficientnet_model = Model(inputs=base_model.input, outputs=output)

# Compile the model
efficientnet_model.compile(optimizer=Adam(learning_rate=0.0001),
                           loss='binary_crossentropy',
                           metrics=['accuracy'])

# Train the model
efficientnet_history = efficientnet_model.fit(
    train_data,
    epochs=10,
    validation_data=test_data
)


16705208/16705208 ━━━━━━━━━━━━━━━━━━━━ 16s 1us/step
Epoch 1/10
58/58 ━━━━━━━━━━━━━━━━━━━━ 67s 902ms/step - accuracy: 0.5123 - loss: 0.6999 - val_accuracy: 0.5000 - val_loss: 0.6934
Epoch 2/10
58/58 ━━━━━━━━━━━━━━━━━━━━ 47s 818ms/step - accuracy: 0.5135 - loss: 0.7034 - val_accuracy: 0.5000 - val_loss: 0.6947
Epoch 3/10
58/58 ━━━━━━━━━━━━━━━━━━━━ 48s 821ms/step - accuracy: 0.4832 - loss: 0.7037 - val_accuracy: 0.5000 - val_loss: 0.6935
Epoch 4/10
58/58 ━━━━━━━━━━━━━━━━━━━━ 46s 795ms/step - accuracy: 0.5078 - loss: 0.6958 - val_accuracy: 0.5000 - val_loss: 0.6934
Epoch 5/10
58/58 ━━━━━━━━━━━━━━━━━━━━ 47s 816ms/step - accuracy: 0.5023 - loss: 0.6971 - val_accuracy: 0.5000 - val_loss: 0.6938
Epoch 6/10
58/58 ━━━━━━━━━━━━━━━━━━━━ 50s 863ms/step - accuracy: 0.4917 - loss: 0.6949 - val_accuracy: 0.5000 - val_loss: 0.6964
Epoch 7/10
58/58 ━━━━━━━━━━━━━━━━━━━━ 49s 844ms/step - accuracy: 0.4772 - loss: 0.6981 - val_accuracy: 0.4717 - val_loss: 0.6932
Epoch 8/10
58/58 ━━━━━━━━━━━━━━━━━━━━ 85s 892

In [7]:
# Load pre-trained MobileNetV2
base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(150, 150, 3))
base_model.trainable = False  # Freeze the base model

# Add custom layers
x = base_model.output
x = tf.keras.layers.GlobalAveragePooling2D()(x)
x = Dense(128, activation='relu')(x)
x = Dropout(0.5)(x)
output = Dense(1, activation='sigmoid')(x)

mobilenet_model = Model(inputs=base_model.input, outputs=output)

# Compile the model
mobilenet_model.compile(optimizer=Adam(learning_rate=0.0001),
                        loss='binary_crossentropy',
                        metrics=['accuracy'])

# Train the model
mobilenet_history = mobilenet_model.fit(
    train_data,
    epochs=100,
    validation_data=test_data
)

C:\Users\hamza\AppData\Local\Temp\ipykernel_8040\3810991184.py:2: UserWarning: `input_shape` is undefined or non-square, or `rows` is not in [96, 128, 160, 192, 224]. Weights for input shape (224, 224) will be loaded as the default.
  base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(150, 150, 3))


Epoch 1/100
58/58 ━━━━━━━━━━━━━━━━━━━━ 49s 729ms/step - accuracy: 0.5907 - loss: 0.7779 - val_accuracy: 0.7761 - val_loss: 0.4443
Epoch 2/100
58/58 ━━━━━━━━━━━━━━━━━━━━ 40s 684ms/step - accuracy: 0.7343 - loss: 0.5402 - val_accuracy: 0.7870 - val_loss: 0.4074
Epoch 3/100
58/58 ━━━━━━━━━━━━━━━━━━━━ 42s 715ms/step - accuracy: 0.7835 - loss: 0.4624 - val_accuracy: 0.8261 - val_loss: 0.3661
Epoch 4/100
58/58 ━━━━━━━━━━━━━━━━━━━━ 40s 678ms/step - accuracy: 0.8025 - loss: 0.4278 - val_accuracy: 0.8630 - val_loss: 0.3330
Epoch 5/100
58/58 ━━━━━━━━━━━━━━━━━━━━ 40s 686ms/step - accuracy: 0.8039 - loss: 0.4265 - val_accuracy: 0.8217 - val_loss: 0.3524
Epoch 6/100
58/58 ━━━━━━━━━━━━━━━━━━━━ 39s 678ms/step - accuracy: 0.8290 - loss: 0.4020 - val_accuracy: 0.8283 - val_loss: 0.3429
Epoch 7/100
58/58 ━━━━━━━━━━━━━━━━━━━━ 39s 673ms/step - accuracy: 0.8216 - loss: 0.4005 - val_accuracy: 0.8478 - val_loss: 0.3258
Epoch 8/100
58/58 ━━━━━━━━━━━━━━━━━━━━ 39s 664ms/step - accuracy: 0.8340 - loss: 0.3780 - 

In [8]:
# Evaluate CNN
cnn_loss, cnn_accuracy = cnn_model.evaluate(test_data)
print(f"CNN Accuracy: {cnn_accuracy * 100:.2f}%")

# Evaluate MobileNetV2
mobilenet_loss, mobilenet_accuracy = mobilenet_model.evaluate(test_data)
print(f"MobileNetV2 Accuracy: {mobilenet_accuracy * 100:.2f}%")

15/15 ━━━━━━━━━━━━━━━━━━━━ 3s 172ms/step - accuracy: 0.5899 - loss: 0.8052
CNN Accuracy: 59.13%
15/15 ━━━━━━━━━━━━━━━━━━━━ 5s 361ms/step - accuracy: 0.8617 - loss: 0.2953
MobileNetV2 Accuracy: 87.17%


In [23]:
def ensemble_predict(models, img_path):
    from tensorflow.keras.preprocessing import image
    img = image.load_img(img_path, target_size=(150, 150))
    img_array = image.img_to_array(img) / 255.0
    img_array = np.expand_dims(img_array, axis=0)
    
    predictions = [model.predict(img_array)[0][0] for model in models]
    final_prediction = np.mean(predictions)
    
    if final_prediction > 0.5:
        print(f"The car is damaged (Confidence: {final_prediction * 100:.2f}%)")
    else:
        print(f"The car is NOT damaged (Confidence: {(1 - final_prediction) * 100:.2f}%)")

# Use ensemble
models = [mobilenet_model]
ensemble_predict(models, "dataset/test/damaged/0011.JPEG")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 77ms/step
The car is damaged (Confidence: 78.60%)


In [24]:
# Save models
mobilenet_model.save('mobilenet_model.h5')

# Load models
from tensorflow.keras.models import load_model
mobilenet_model = load_model('mobilenet_model.h5')
